In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal
# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType
from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm
from comau_msgs.srv import SetIO, SetIORequest
rospy.init_node('TUM_pilot')
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()
def cancel(cl):
    cl.cancel_all_goals()
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

[rosout] [/TUM_pilot] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/TUM_pilot] connected to Matlab
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_42785 with NodeURI http://192.168.2.87:45243/
[rosout] [/TUM_pilot] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


## Tool parameter

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
#cobot.change_tool("nothing")
# cobot.change_tool("screwdriver")
#cobot.change_tool("wsg_50_dsa")
#cobot.change_tool("wsg_50") #Pneumatic Finger Gripper
cobot.change_tool("shaftgrinder")
# cobot.change_tool("vacuum")

## Skill actions

In [ ]:
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

##### Backout

In [ ]:
cancel(shaftgrinder_client)
cancel(unscrewing_client)
cancel(tool_change_client)
cancel(vacuum_client)
cancel(pgrip_client)
cancel(grasping_client)

In [ ]:
cobot.stop()

In [ ]:
# go up
cartesian_difference = [0, 0,0.1]
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.05)

In [ ]:
# arbitrary
cartesian_difference = [0, -0,0]
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.05)

In [ ]:
# go neutral
q_neutral = np.r_[1.34109, 0.09328, -1.44435, -0.01627, 0.67686, -0.00009]
cobot.move_to_joint_pose(q_neutral, stochastic=False)

In [ ]:
# screwdriver1 = np.r_[ 0.86289,  0.54018, -1.66603,  1.60357, -1.63773,  0.63349]
# screwdriver2 = np.r_[ 0.96828,  0.17584, -1.5306 , -1.55063,  1.62805, -3.01406]
# cobot.move_to_joint_pose(screwdriver2, stochastic=False)

In [ ]:
# go calib pose
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

### Open clamp

In [ ]:
from comau_msgs.srv import SetIO, SetIORequest
srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
srv(SetIORequest(pin=300, state=True)) 
rospy.sleep(1.0)
srv(SetIORequest(pin=300, state=False))

### Open tool changer

In [ ]:
cobot.close_tool_changer()

In [ ]:
cobot.open_tool_changer()

## Tool change

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
tc_goal = ChangeToolGoal()
# set as needed

tc_goal.new_tool.type = tc_goal.new_tool.NONE
# tc_goal.new_tool.type = tc_goal.new_tool.WSG_50 #pneumatic gripper
tc_goal.new_tool.type = tc_goal.new_tool.WSG_50_DSA
# tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
#tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
# tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER

In [ ]:
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
tool_change_client.send_goal(tc_goal)

In [ ]:
cancel(tool_change_client)

# Gripper reset

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.gripper.reset()

In [ ]:
cobot.gripper.width

In [ ]:
#Close
cobot.gripper.set_vel(-0.08,si=True)

In [ ]:
#Stop
cobot.gripper.set_vel(0.0, si=True)

In [ ]:
#Open
cobot.gripper.set_vel(0.08,si=True)

# We crashed, better restart FT sensor?

In [ ]:
cobot.FT.f #Raw force values

In [ ]:
cobot.B_F_msr #Force values transformed

In [ ]:
cobot.FT.reset()

In [ ]:
cobot.FT.reset_bias()